In [1]:
import pickle
import csv
import math
import pandas as pd
import numpy as np
import os

In [2]:
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem

RDKit WARNING: [12:24:38] Enabling RDKit 2019.09.3 jupyter extensions


In [3]:
from sklearn.preprocessing import KernelCenterer

In [4]:
from process_dataset.process_DB import get_DB

# Benoit's data

In [7]:
benoit_dir = "/Users/matthieu/ownCloud/CFTR_PhD/Chemogenomics/Benoit/passage_de_relai/CFTR_PROJECT/data/NNdti_S0h/"

In [21]:
old_dict_ligand = pickle.load(open(benoit_dir + 
'NNdti_S0h_dict_DBid2smiles.data', 'rb'))
old_dict_target = pickle.load(open(benoit_dir + 
'NNdti_S0h_dict_uniprot2fasta.data', 'rb'))

old_intMat = np.load(benoit_dir + 'NNdti_S0h_intMat.npy')
    
old_dict_ind2mol = pickle.load(open(benoit_dir + 
'NNdti_S0h_dict_ind2mol.data', 'rb'))
old_dict_mol2ind = pickle.load(open(benoit_dir + 
'NNdti_S0h_dict_mol2ind.data', 'rb'))
old_dict_prot2ind = pickle.load(open(benoit_dir + 
'NNdti_S0h_dict_prot2ind.data', 'rb'))
old_dict_ind2prot = pickle.load(open(benoit_dir + 
'NNdti_S0h_dict_ind2prot.data', 'rb'))

In [9]:
old_K_mol_norm = pickle.load(open(benoit_dir + 'NNDti_S0h_Kmol_norm.data', 'rb'))
old_K_mol = pickle.load(open(benoit_dir + 'NNDti_S0h_Kmol.data', 'rb'))
# old_K_prot = pickle.load(open(benoit_dir + 'NNDti_S0h_Kprot_norm.data', 'rb'))

# Matthieu's data

In [10]:
os.getcwd()

'/Users/matthieu/ownCloud/Programming/DTI_prediction'

In [11]:
root = '../CFTR_PROJECT/'

In [12]:
DB_version = "drugbank_v5.1.1"
DB_type = "S0h"
process_name = "Benoit_NNdti"

In [13]:
# data_dir variable 
data_dir = 'data/' + DB_version + '/' + process_name + '_' + DB_type + '/'
# pattern_name variable
pattern_name = process_name + '_' + DB_type

In [14]:
raw_data_dir = 'data/' + DB_version + '/raw/'

In [74]:
dict_ligand = pickle.load(open(root + data_dir + pattern_name + '_dict_DBid2smiles.data', 'rb'))
dict_target = pickle.load(open(root + data_dir + pattern_name + '_dict_uniprot2fasta.data', 'rb'))

list_interactions = pickle.load(open(root + data_dir + pattern_name + '_list_interactions.data', 'rb'))
intMat = np.load(root + data_dir + pattern_name + '_intMat.npy')
    
dict_ind2mol = pickle.load(open(root + data_dir + pattern_name + '_dict_ind2mol.data', 'rb'))
dict_mol2ind = pickle.load(open(root + data_dir + pattern_name + '_dict_mol2ind.data', 'rb'))
dict_prot2ind = pickle.load(open(root + data_dir + pattern_name + '_dict_prot2ind.data', 'rb'))
dict_ind2prot = pickle.load(open(root + data_dir + pattern_name + '_dict_ind2prot.data', 'rb'))

In [82]:
dict_ind2mol.values()

dict_values(['DB08392', 'DB07053', 'DB09013', 'DB11967', 'DB07314', 'DB04036', 'DB01431', 'DB03381', 'DB02740', 'DB07899', 'DB09238', 'DB07276', 'DB01067', 'DB11085', 'DB08839', 'DB07435', 'DB13167', 'DB02935', 'DB00586', 'DB04768', 'DB00984', 'DB08385', 'DB04098', 'DB08351', 'DB07288', 'DB00128', 'DB00829', 'DB02548', 'DB06900', 'DB02902', 'DB03126', 'DB01946', 'DB03128', 'DB01185', 'DB08088', 'DB00148', 'DB04828', 'DB00275', 'DB03337', 'DB00844', 'DB03849', 'DB01028', 'DB13929', 'DB07324', 'DB00680', 'DB03753', 'DB00129', 'DB02318', 'DB01837', 'DB04843', 'DB00824', 'DB00863', 'DB06732', 'DB04394', 'DB06176', 'DB02562', 'DB02258', 'DB11644', 'DB02936', 'DB09089', 'DB07498', 'DB02170', 'DB08927', 'DB08037', 'DB07913', 'DB08633', 'DB11201', 'DB02761', 'DB04832', 'DB12582', 'DB04079', 'DB00254', 'DB13872', 'DB02008', 'DB03085', 'DB01189', 'DB02393', 'DB00484', 'DB03924', 'DB07916', 'DB04089', 'DB04254', 'DB00188', 'DB09265', 'DB03902', 'DB07216', 'DB05964', 'DB11386', 'DB04590', 'DB02726

In [16]:
K_mol_norm = pickle.load(open(root + data_dir + pattern_name + '_Kmol_norm.data', 'rb'))
K_mol = pickle.load(open(root + data_dir + pattern_name + '_Kmol.data', 'rb'))
# K_prot = pickle.load(open(benoit_dir + '_Kprot_norm.data', 'rb'))

# Comparaison

## Dictionaries

In [25]:
def compare_two_dictionaries(x, y):
    shared_items = {k: x[k] for k in x if k in y and x[k] == y[k]}
    print(len(shared_items))

In [26]:
def diff_items_two_dictionaries(x, y):
    diff_items = {k: x[k] for k in x if k in y and x[k] != y[k]}
    print(diff_items)

In [28]:
compare_two_dictionaries(old_dict_ligand, dict_ligand)
diff_items_two_dictionaries(old_dict_ligand, dict_ligand)

4837
{}


## Numpy arrays

In [30]:
np.array_equal(old_intMat, intMat)

True

In [44]:
diff_K_mol_norm = K_mol_norm - old_K_mol_norm

In [66]:
np.sum(diff_K_mol_norm >= 0.05)

2

In [65]:
np.sum(K_mol_norm >= 0.03)

5159403

In [56]:
5114254 - (1057 * 4837)

1545

In [58]:
diff_K_mol_norm[1057, 1545]

0.053557818168606

In [67]:
a = 'mkdir nwdir'

b = os.popen(a, 'r', 1)

In [68]:
print(b)